# This notebook allow users to access NOAA Coastal Ocean ReAnalysis (CORA) data that is stored on NOAA's Open Data Dissemination (NODD) service. The data is will be loaded as an xarray dataset. Here, the water level data is extracted at the nearest model node to user provided geographic coordinates and the data is displayed in a time series plot.

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask
import intake
import xarray as xr
import holoviews as hv
import geoviews as gv
import hvplot.xarray
import holoviews.operation.datashader as dshade
from bokeh.models import DatetimeTickFormatter, HoverTool
import cmocean

hv.extension('bokeh')

C:\Users\John.Ratcliff\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


**Access the data on the NODD and initialize the available CORA datasets.** 

*This accesses a .yml file located on the NODD that shows which CORA output files are available to import.*

In [2]:

catalog = intake.open_catalog("s3://noaa-nos-cora-pds/CORA_intake.yml",storage_options={'anon':True})
list(catalog)


['CORA-V1-fort.63',
 'CORA-V1-maxele.63',
 'CORA-V1-fort.64',
 'CORA-V1-500m-grid-1979-2022']

**The CORA-V1-fort.63 file contains hourly water levels at the model mesh nodes for 1979-2022. The CORA-V1-maxele.63 file contains the maximum water level for the entire 1979-2022 period modeled at each of the model nodes. The CORA-V1-fort.64 file contains the hourly current velocities (u and v) at each of the model nodes for 1979-2022. The CORA-V1-500m-grid-1979-2022 file contains hourly water levels that have been interpolated from the model mesh nodes to uniformly space 500-meter grid nodes.**

## Using to_dask() with one of the catalog files will create an xarray dataset that contains the model grid information along with the time and water level ('zeta') variables.

**This is the dataset for the hourly water levels at all model nodes.**

In [3]:
ds = catalog["CORA-V1-fort.63"].to_dask()
ds

<xarray.Dataset>
Dimensions:      (mesh: 1, node: 1813443, nele: 3564104, nvertex: 3, nbou: 186,
                  nope: 1, neta_dim: 186, nvel_dim: 62972, time: 385704)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2022-12-31T23:00:00
    x            (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
    y            (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
Dimensions without coordinates: mesh, node, nele, nvertex, nbou, nope,
                                neta_dim, nvel_dim
Data variables: (12/14)
    adcirc_mesh  (mesh) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    depth        (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
    element      (nele, nvertex) int32 dask.array<chunksize=(3564104, 3), meta=np.ndarray>
    ibtype       (nbou) int32 dask.array<chunksize=(186,), meta=np.ndarray>
    ibtypee      (nope) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    max_nvdll    float64 ...
    ...           ...
    nbvv         (nvel_dim) int32 dask.array<chunksize=(62972,), meta=np.ndarray>
    neta         float64 ...
    nvdll        (nope) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    nvel         float64 ...
    nvell        (nbou) int32 dask.array<chunksize=(186,), meta=np.ndarray>
    zeta         (time, node) float64 dask.array<chunksize=(240, 160000), meta=np.ndarray>
Attributes: (12/49)
    Conventions:        UGRID-0.9.0
    NCO:                netCDF Operators version 4.7.5 (Homepage = http://nco...
    a00:                0.0
    agrid:              NOMAD mesh v1e MSL - fixed bahamas bathy
    b00:                1.0
    c00:                0.0
    ...                 ...
    slam0:              265.5
    source:             RENCI_HPC
    statim:             0.0
    tau0:               -3.0
    title:              NOAA-Reanalysis-Post
    version:            noaa.stofs.2d.glo.v1.1.0r2-21-g7052115

## Create function to find the nearest node to a given set of coordinates.

In [4]:
# find the indices of the points in (x,y) closest to the points in (xi,yi)
def nearxy(x,y,xi,yi):
    ind = np.ones(len(xi),dtype=int)
    for i in range(len(xi)):
        dist = np.sqrt((x-xi[i])**2+(y-yi[i])**2)
        ind[i] = dist.argmin()
    return ind

**Provide coordinates.**

In [5]:

lat = 32.751
lon = -79.867

**Run the function.**

In [6]:
ind = nearxy(ds['x'].values,ds['y'].values,[lon], [lat])

**Extract data for specified time and create plot using hvplot.**

In [8]:
start="2018-09-10"
end="2018-09-18"

tickfmt = DatetimeTickFormatter(years="%m/%d/%Y", months="%m/%d/%Y", days = '%m/%d/%Y', hourmin = '%H:%M')
tooltips = [
    ("time", "@time{%F %T}"),
    ("water level", "@zeta"),
]
hover = HoverTool(tooltips=tooltips,formatters={
        '@time': 'datetime'})

ds['zeta'][:,ind].sel(time=slice(start, end)).hvplot(x='time', grid=True, xformatter=tickfmt, tools=[hover])

:Curve   [time]   (zeta)